<a href="https://colab.research.google.com/github/GianmarcoLattaruolo/Vision_Project/blob/main/MappaGeografica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Classes proportion**

In [1]:
# with this line we can check if we are in colab or not
import os
import sys
from pathlib import Path
in_colab = 'google.colab' in sys.modules
print("are we in Colab?:",in_colab)

cwd = Path(os.getcwd())
if in_colab:
    from google.colab import drive
    drive.mount('/gdrive')
    !pip install -q condacolab
    import condacolab
    condacolab.install()
    os.chdir('/gdrive/MyDrive/GeoEstimation')
    
else:
    #our defult wd in local should be Vision_Project
    if str(cwd)[-14:] == 'Vision_Project':
        os.chdir(cwd / 'GeoEstimation')
    sys.path.append(cwd / 'GeoEstimation')

are we in Colab?: False


In [3]:
import pandas as pd
#the default working directory is geoestimation
r = pd.read_csv(r'\resources\yfcc4k.csv')
n = r.shape[0]
a,b,c = sum(r.LOC==0.0)/n,sum(r.LOC==1.0)/n,sum(r.LOC==2.0)/n
print(a,b,c)

0.4015869517302182 0.1692748512232753 0.392109323341415


## Produce the csv with the proportions
We want a .csv file with the proportions of images labelled as class 0, 1 or 2 in the wrong classified (wrong geolocated) and right classified images, these for all the cell partitionings.

In [77]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
  
# Opening JSON file
#f_original = open('/gdrive/MyDrive/GeoEstimation/resources/im2gps_places365_mapping_h3.json')
f_original = open(r'\resources\im2gps3k_places365_mapping_h3.json')
df_predicted = pd.read_csv(r'\resources\predicted_coordinates3k.csv')
localizable = pd.read_csv(r'\resources\im2gps3k_places365.csv')

display(df_predicted.tail())

original_test = json.load(f_original)
original_test_df = pd.DataFrame.from_dict(original_test, orient='index')
original_test_df.reset_index(inplace=True)
original_test_df = original_test_df.rename({'index': 'IMG_ID', 0: 'coarse', 1: 'middle', 2: 'fine'}, axis=1)
#original_test_df['IMG_ID'] = original_test_df['IMG_ID'].str.replace('.jpg', '')

display(original_test_df.head())
smaller_loc = localizable.loc[:, ['IMG_ID', 'LOC']]
display(smaller_loc.head())
loc_cell_df = smaller_loc.merge(on='IMG_ID',right=original_test_df)
loc_cell_df.head()
display(df_predicted.head())

c = df_predicted[df_predicted.p_key=='coarse']
c = c[['img_id','pred_class']]
c.columns = ['IMG_ID','coarse_pred']
m = df_predicted[df_predicted.p_key=='middle']
m = m[['img_id','pred_class']]
m.columns = ['IMG_ID','middle_pred']
f = df_predicted[df_predicted.p_key=='hierarchy']
f = f[['img_id','pred_class']]
f.columns = ['IMG_ID','fine_pred']
full = c.merge(on='IMG_ID', right = m).merge(on='IMG_ID',right = f)
display(full.head())
loc_cell_df['IMG_ID'] = loc_cell_df['IMG_ID'].str.replace('.jpg', '')
display(loc_cell_df.head())

entire = full.merge(on='IMG_ID', right = loc_cell_df)
display(entire.head())
entire_fine = entire[entire['fine'] == entire['fine_pred'] ]
entire_middle = entire[entire['middle'] == entire['middle_pred']]
entire_coarse = entire[entire['coarse'] == entire['coarse_pred']]
display(entire_fine.head())  
display(entire_middle.head())  
display(entire_coarse.head())  
n = entire.shape[0]

original = np.asarray([sum(entire.LOC == 0.0),sum(entire.LOC == 1.0),sum(entire.LOC == 2.0)] )
#original = original/n
data = np.zeros((7,3))
data[0,:] = original

fine_prop = np.asarray([sum(entire_fine.LOC == 0.0),sum(entire_fine.LOC == 1.0),sum(entire_fine.LOC == 2.0)])
den = entire_fine.shape[0]
#fine_prop = fine_prop/den
data[5,:] = fine_prop
data[6,:] = original - data[5,:]

middle_prop = np.asarray([sum(entire_middle.LOC == 0.0),sum(entire_middle.LOC == 1.0),sum(entire_middle.LOC == 2.0)])
den = entire_middle.shape[0]
#middle_prop = middle_prop/den
data[3,:] = middle_prop
data[4,:] = original - data[3,:]

coarse_prop = np.asarray([sum(entire_coarse.LOC == 0.0),sum(entire_coarse.LOC == 1.0),sum(entire_coarse.LOC == 2.0)])
den = entire_coarse.shape[0]
#coarse_prop = coarse_prop/den
data[1,:] = coarse_prop
data[2,:] = original - data[1,:]

proportion = pd.DataFrame(data = data, index = ['original','right_coarse','wrong_coarse','right_middle','wrong_middle','right_fine','wrong_fine'], columns= ['c0','c1','c2'])
display(proportion.head(10))
proportion  =proportion.divide(other = proportion.sum(axis = 1), axis = 0)
display(proportion.head(10))

proportion.to_csv(r'\resources\proportion_3ktest.csv')


,img_id,p_key,pred_class,pred_lat,pred_lng
11983,99078324_9d8e7a3975_28_44972214@N00,hierarchy,5473,32.843647,-117.275940
11984,99101189_a518846673_43_50517642@N00,hierarchy,5456,-17.110973,-149.309921
11985,99115715_1bdd45ca71_19_50517642@N00,hierarchy,497,23.571377,119.617821
11986,995674057_2360a67996_1182_98545448@N00,hierarchy,4730,51.696236,-1.802698
11987,995764493_f8128f77c1_1226_98545448@N00,hierarchy,4730,51.696236,-1.802698


,IMG_ID,coarse,middle,fine
0,1000269685_e60e9cdfb4_1125_78841376@N00.jpg,2323,3970,5249
1,1000304467_1a75a200b1_1296_78841376@N00.jpg,2323,3970,5249
2,1001048550_8e4b47d165_1051_78841376@N00.jpg,2323,3970,5249
3,1005977048_5ccf8b05d3_1201_91728102@N00.jpg,3104,6469,10864
4,1008804117_ce4e6fef8a_1349_97522422@N00.jpg,1374,1815,2002


,IMG_ID,LOC
0,1000269685_e60e9cdfb4_1125_78841376@N00.jpg,0.0
1,1000304467_1a75a200b1_1296_78841376@N00.jpg,1.0
2,1001048550_8e4b47d165_1051_78841376@N00.jpg,0.0
3,1005977048_5ccf8b05d3_1201_91728102@N00.jpg,2.0
4,1008804117_ce4e6fef8a_1349_97522422@N00.jpg,0.0


,img_id,p_key,pred_class,pred_lat,pred_lng
0,1000269685_e60e9cdfb4_1125_78841376@N00,coarse,192,32.730507,-117.214523
1,1000304467_1a75a200b1_1296_78841376@N00,coarse,192,32.730507,-117.214523
2,1001048550_8e4b47d165_1051_78841376@N00,coarse,317,-33.859810,151.215057
3,1005977048_5ccf8b05d3_1201_91728102@N00,coarse,617,34.785538,128.915405
4,1008804117_ce4e6fef8a_1349_97522422@N00,coarse,473,40.741028,-119.243790


,IMG_ID,coarse_pred,middle_pred,fine_pred
0,1000269685_e60e9cdfb4_1125_78841376@N00,192,3350,6600
1,1000304467_1a75a200b1_1296_78841376@N00,192,3970,5249
2,1001048550_8e4b47d165_1051_78841376@N00,317,2827,4481
3,1005977048_5ccf8b05d3_1201_91728102@N00,617,105,326
4,1008804117_ce4e6fef8a_1349_97522422@N00,473,514,527


...

,IMG_ID,coarse_pred,middle_pred,fine_pred,LOC,coarse,middle,fine
1,1000304467_1a75a200b1_1296_78841376@N00,192,3970,5249,1.0,2323,3970,5249
24,103577701_b68e15492e_32_73293249@N00,1080,4010,5316,2.0,1080,4010,5316
38,104146945_2722f724b3_19_14542551@N00,1023,1231,2822,1.0,1023,1231,2822
39,1044041591_512737f6a3_1182_9729483@N02,1439,1929,2137,1.0,1439,1929,2137
40,104419192_8551684362_41_64825499@N00,14,13,5455,2.0,14,13,5455


,IMG_ID,coarse_pred,middle_pred,fine_pred,LOC,coarse,middle,fine
1,1000304467_1a75a200b1_1296_78841376@N00,192,3970,5249,1.0,2323,3970,5249
9,1021054395_ca7cfe55b3_1061_7132415@N03,1660,2355,8412,0.0,1660,2355,6257
22,103574519_62331b624f_24_73293249@N00,79,80,79,2.0,79,80,1838
23,103575522_cc6a522170_34_73293249@N00,79,80,79,2.0,79,80,1838
24,103577701_b68e15492e_32_73293249@N00,1080,4010,5316,2.0,1080,4010,5316


,IMG_ID,coarse_pred,middle_pred,fine_pred,LOC,coarse,middle,fine
9,1021054395_ca7cfe55b3_1061_7132415@N03,1660,2355,8412,0.0,1660,2355,6257
11,1029739337_ff06001d12_1361_7132415@N03,1660,835,923,0.0,1660,2355,9259
22,103574519_62331b624f_24_73293249@N00,79,80,79,2.0,79,80,1838
23,103575522_cc6a522170_34_73293249@N00,79,80,79,2.0,79,80,1838
24,103577701_b68e15492e_32_73293249@N00,1080,4010,5316,2.0,1080,4010,5316


,c0,c1,c2
original,791.0,663.0,1510.0
right_coarse,58.0,94.0,404.0
wrong_coarse,733.0,569.0,1106.0
right_middle,45.0,77.0,341.0
wrong_middle,746.0,586.0,1169.0
right_fine,33.0,68.0,278.0
wrong_fine,758.0,595.0,1232.0


,c0,c1,c2
original,0.266869,0.223684,0.509447
right_coarse,0.104317,0.169065,0.726619
wrong_coarse,0.304402,0.236296,0.459302
right_middle,0.097192,0.166307,0.736501
wrong_middle,0.298281,0.234306,0.467413
right_fine,0.087071,0.179420,0.733509
wrong_fine,0.293230,0.230174,0.476596
